In [1]:
import sys
# from product import *
from data_util.product import *

import pandas as pd
import random
import time
import re

import os
import hashlib
import struct
import subprocess
import collections
import tensorflow as tf
from tensorflow.core.example import example_pb2
import nltk

import sys
import shutil
import tqdm
import random

from copy import deepcopy
# from product import *

VOCAB_SIZE = 50000
CHUNK_SIZE = 1000  # num examples per chunk, for the chunked data

D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\pyth

# Key word Attention DataSet 讀取

In [2]:
# from data_util.config import *
# _, category1, category2, _ = DVD_Player().getAttr()
_,category1,category2,_ = Cameras().getAttr()
# _,category1,category2,_ = Cell_Phones().getAttr()
# _,category1,category2,_ = GPS().getAttr()
# _,category1,category2,_ = Keyboards().getAttr()
# category1,category2 = config.category1 , config.category2

xlsx_path = "XLSX/category/%s_%s_key.xlsx"%(category1,category2)
# df.to_csv(csv_path) #默认dt是DataFrame的一个实例，参数解释如下
# key_train_df.to_excel(csv_path, encoding='utf8')
orign_key_df = pd.read_excel(xlsx_path)
print(xlsx_path + " Read finished")
len(orign_key_df)

orign_key_df['bert_review'] = '' 
orign_key_df['bert_summary'] = ''
orign_key_df['bert_review_len'] = 0
orign_key_df['bert_summary_len'] = 0

orign_key_df.head()


XLSX/category/Electronics_Cameras_key.xlsx Read finished


,Unnamed: 0,review_ID,review,summary,big_categories,small_categories,lemm_review,lemm_summary,lemm_review_len,lemm_summary_len,overall,vote,total_keyword,FOP_sents,total_mention_features,bert_review,bert_summary,bert_review_len,bert_summary_len
0,0,1356566400,this book was almost as small a pamphlet . for...,too small .,Electronics,Camera Cases,this book was almost as small a pamphlet . \nf...,<s> too small </s> \n,45,6,1,2,"pamphlet as small,pamphlet small",this book was almost as small a pamphlet .,small pamphlet book gift expect intend emphasi...,,,0,0
1,1,1325721600,this book is a collectors item and i love they...,wonderful book a keepsake,Electronics,Camera Cases,this book is a collector item and i love they ...,<s> wonderful book a keepsake </s> \n,54,8,5,9,"item great,item wonderful,quality great,nostal...",\ngreat quality though and a wonderful nostalg...,collector item book high copy price price buy ...,,,0,0
2,2,951782400,i would like to say the the cannon elph37oz is...,elph 37oz is the best .,Electronics,APS Cameras,i would like to say the the cannon elph37 oz i...,<s> elph 37 oz is the best </s> \n,58,10,5,51,"money extra,camera cheap,camera inferior",\npay the extra money and get this one rather ...,money cannon camera decide buy picture camera ...,,,0,0
3,3,985910400,i have had and used q 370z for over a year and...,a good all around aps camera,Electronics,APS Cameras,i have have and use q 370z for over a year and...,<s> a good all around aps camera </s> \n,137,10,5,13,"light low,eye red room increase slrs lense,len...",\nyes i sometimes get red eye in low light but...,year photographer snapshot lens eye close flas...,,,0,0
4,4,967075200,i used this camera once and it took relatively...,terrible camera worse customer service,Electronics,APS Cameras,i use this camera once and it take relatively ...,<s> terrible camera worse customer service </...,94,9,1,48,memory internal camera service camera send mon...,\nhowever on my next trip the camera internal ...,picture camera memory camera trip malfunction ...,,,0,0


# Key word load

In [3]:
fn = 'FOP-View/%s_%s_keywords2.txt' % (category1, category2)
print('load %s keywords...' % (fn))
total_keywords = set()
with open(fn, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        k, v = line.split(":")
        total_keywords.add(k)

load FOP-View/Electronics_Cameras_keywords2.txt keywords...


# Total Opinion

In [4]:
opinion_lexicon = {}
for filename in os.listdir('opinion-lexicon-English/'):      
    if "txt" not in filename: continue
    print(filename)
    with open('opinion-lexicon-English/'+filename,'r') as f_input:
        lexion = []
        for line in f_input:
            if line.startswith(";"):
                continue
            word = line.replace("\n","")
            if word != "" : lexion.append(word)
        pos = filename.replace(".txt","")
        opinion_lexicon[pos] = lexion

opinion_lexicon["total-words"] = opinion_lexicon["negative-words"] + opinion_lexicon["positive-words"]
print("total-words 已取得")

negative-words.txt
positive-words.txt
total-words 已取得


In [5]:
# tokenizer 裡頭的字典資訊

In [6]:
from pytorch_pretrained_bert import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True, do_basic_tokenize=True)
vocab = tokenizer.vocab # word_to_id
print("字典大小：", len(vocab))

字典大小： 30522


# Bert-Summary 資料清理

In [7]:
import spacy
from collections import Counter
import en_core_web_sm
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

nlp = en_core_web_sm.load()

def isnumber(aString):
    try:
        float(aString)
        return True
    except:
        return False
    
def create_custom_tokenizer(nlp):
    prefix_re = re.compile(r'[0-9]\.')
    return Tokenizer(nlp.vocab, prefix_search=prefix_re.search)

nlp.tokenizer = create_custom_tokenizer(nlp)

alphbet_stopword = ['b','c','d','e','f','g','h','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

In [8]:
def compose_summary(x):
    x = x.replace("\n"," ")
    #x = x.replace("\n","").replace("</s>","").replace("<s>","")
    #x = "<s>" + x + "</s>"  
    tokens = [str(token) for token in x.split(" ") if (" " not in str(token)) and \
                   #(str(token).isalpha()) and \
                  (len(str(token)) > 1)   ]
    # print(tokens)
    #return " ".join(tokens),tokens
    
    newtokens = []
    for token in tokens:
        if (isnumber(token) or len(token) == 1 or token == ".") and (token not in alphbet_stopword):
            newtokens.append(token)
        else:
            token = token.replace("."," . ")
            sub_tokens = token.split(" ")
            sub_tokens = [t for t in sub_tokens if t != "" and t not in alphbet_stopword]
            newtokens.extend(sub_tokens)
    
#     dot_tokens = [token for token in newtokens if (token[0] == "." or token[-1] == ".") and (len(token)>1)]
#     if len(dot_tokens) > 0 :print(dot_tokens)
        
    return " ".join(newtokens).replace(' . . ',' . '),newtokens
    

def bert_compose_summary(newtokens):
#     x = x.replace("\n","").replace("</s>","").replace("<s>","")
#     x = "<s>" + x + "</s>"  
#     tokens = [str(token) for token in nlp(x) if (" " not in str(token)) and \
#                   (str(token).isalpha()) and \
#                   (len(str(token)) > 1)   ]
    
#     newtokens = []
#     for token in tokens:
#         if (isnumber(token) or len(token) == 1 or token == ".") and (token not in alphbet_stopword):
#             newtokens.append(token)
#         else:
#             token = token.replace("."," . ")
#             sub_tokens = token.split(" ")
#             sub_tokens = [t for t in sub_tokens if t != "" and t not in alphbet_stopword]
#             newtokens.extend(sub_tokens)

#     dot_tokens = [t for t in newtokens if ("." in t) and (len(t) > 1) ]
#     if len(dot_tokens) > 0 :print(dot_tokens)
    newtokens = [t for t in newtokens if t not in ["<s>","</s>"]]
    newtokens = ['[CLS]'] + tokenizer.tokenize(" ".join(newtokens)) + ['[SEP]']
    return " ".join(newtokens)

'''
def calc_summary_len(x):
#     tokens = [token for token in nlp(x)]
#     print(tokens)
#     print([len(t) for t in tokens])
#     return len(tokens)
    return len(x.split(" "))

nlp.tokenizer = create_custom_tokenizer(nlp)

orign_key_df['lemm_summary'] = orign_key_df['lemm_summary'].apply(compose_summary)
orign_key_df['lemm_summary_len'] = orign_key_df['lemm_summary'].apply(calc_summary_len)


amount = len(orign_key_df)
print('Total data : %s'%(amount))

orign_key_df.head()
'''
orign_key_df.head()

,Unnamed: 0,review_ID,review,summary,big_categories,small_categories,lemm_review,lemm_summary,lemm_review_len,lemm_summary_len,overall,vote,total_keyword,FOP_sents,total_mention_features,bert_review,bert_summary,bert_review_len,bert_summary_len
0,0,1356566400,this book was almost as small a pamphlet . for...,too small .,Electronics,Camera Cases,this book was almost as small a pamphlet . \nf...,<s> too small </s> \n,45,6,1,2,"pamphlet as small,pamphlet small",this book was almost as small a pamphlet .,small pamphlet book gift expect intend emphasi...,,,0,0
1,1,1325721600,this book is a collectors item and i love they...,wonderful book a keepsake,Electronics,Camera Cases,this book is a collector item and i love they ...,<s> wonderful book a keepsake </s> \n,54,8,5,9,"item great,item wonderful,quality great,nostal...",\ngreat quality though and a wonderful nostalg...,collector item book high copy price price buy ...,,,0,0
2,2,951782400,i would like to say the the cannon elph37oz is...,elph 37oz is the best .,Electronics,APS Cameras,i would like to say the the cannon elph37 oz i...,<s> elph 37 oz is the best </s> \n,58,10,5,51,"money extra,camera cheap,camera inferior",\npay the extra money and get this one rather ...,money cannon camera decide buy picture camera ...,,,0,0
3,3,985910400,i have had and used q 370z for over a year and...,a good all around aps camera,Electronics,APS Cameras,i have have and use q 370z for over a year and...,<s> a good all around aps camera </s> \n,137,10,5,13,"light low,eye red room increase slrs lense,len...",\nyes i sometimes get red eye in low light but...,year photographer snapshot lens eye close flas...,,,0,0
4,4,967075200,i used this camera once and it took relatively...,terrible camera worse customer service,Electronics,APS Cameras,i use this camera once and it take relatively ...,<s> terrible camera worse customer service </...,94,9,1,48,memory internal camera service camera send mon...,\nhowever on my next trip the camera internal ...,picture camera memory camera trip malfunction ...,,,0,0


In [9]:
from tqdm import tqdm
# 非符號alpha word重疊數
with tqdm(total=len(orign_key_df)) as pbar:
    for i ,row in orign_key_df.iterrows():        
        pbar.update(1)
        pbar.set_description("row %s " % (i))

        lemm_summary,newtokens = compose_summary(row['lemm_summary'])
        # bert_summary = bert_compose_summary(newtokens)
        
        orign_key_df.loc[i,'lemm_summary'] = lemm_summary
        # orign_key_df.loc[i,'bert_summary'] = bert_summary
        
        orign_key_df.loc[i,'lemm_summary_len'] = len(lemm_summary.split(" "))       
        # orign_key_df.loc[i,'bert_summary_len'] = len(bert_summary.split(" "))

        
        
amount = len(orign_key_df)
print('Total data : %s'%(amount))
orign_key_df.head()

row 165309 : 100%|██████████| 165310/165310 [1:01:39<00:00, 44.68it/s]

Total data : 165310


,Unnamed: 0,review_ID,review,summary,big_categories,small_categories,lemm_review,lemm_summary,lemm_review_len,lemm_summary_len,overall,vote,total_keyword,FOP_sents,total_mention_features,bert_review,bert_summary,bert_review_len,bert_summary_len
0,0,1356566400,this book was almost as small a pamphlet . for...,too small .,Electronics,Camera Cases,this book was almost as small a pamphlet . \nf...,<s> too small </s>,45,4,1,2,"pamphlet as small,pamphlet small",this book was almost as small a pamphlet .,small pamphlet book gift expect intend emphasi...,,,0,0
1,1,1325721600,this book is a collectors item and i love they...,wonderful book a keepsake,Electronics,Camera Cases,this book is a collector item and i love they ...,<s> wonderful book keepsake </s>,54,5,5,9,"item great,item wonderful,quality great,nostal...",\ngreat quality though and a wonderful nostalg...,collector item book high copy price price buy ...,,,0,0
2,2,951782400,i would like to say the the cannon elph37oz is...,elph 37oz is the best .,Electronics,APS Cameras,i would like to say the the cannon elph37 oz i...,<s> elph 37 oz is the best </s>,58,8,5,51,"money extra,camera cheap,camera inferior",\npay the extra money and get this one rather ...,money cannon camera decide buy picture camera ...,,,0,0
3,3,985910400,i have had and used q 370z for over a year and...,a good all around aps camera,Electronics,APS Cameras,i have have and use q 370z for over a year and...,<s> good all around aps camera </s>,137,7,5,13,"light low,eye red room increase slrs lense,len...",\nyes i sometimes get red eye in low light but...,year photographer snapshot lens eye close flas...,,,0,0
4,4,967075200,i used this camera once and it took relatively...,terrible camera worse customer service,Electronics,APS Cameras,i use this camera once and it take relatively ...,<s> terrible camera worse customer service </s>,94,7,1,48,memory internal camera service camera send mon...,\nhowever on my next trip the camera internal ...,picture camera memory camera trip malfunction ...,,,0,0


# Bert-review 多句合併

In [10]:
from copy import deepcopy
def compose_review(x):
    x = eval(x)
    x = "\n".join(x)
    x = x.replace(".\n"," . ").replace("\n."," . ").replace("\n"," ")
#     x = x.replace("\n"," ")    
    tokens = [str(token) for token in x.split(" ") if (" " not in str(token))and (str(token) == '.' or str(token).isalpha())]
#    tokens = [str(token) for token in x.split(" ") if (" " not in str(token)) ]

    #return " ".join(tokens)
    
    newtokens = []
    for token in tokens:
#         if (len(token) == 1 or token == "."):
        if (isnumber(token) or len(token) == 1 or token == ".")and (token not in alphbet_stopword):
            newtokens.append(token)
#             if (token not in ['a','i']) and (token != "."): print(token)
        else:
#             token = token.replace("."," . ")
            token = token.replace("."," . ")
            sub_tokens = token.split(" ")
            sub_tokens = [t for t in sub_tokens if t != "" and t not in alphbet_stopword]
            if len(sub_tokens) == 0: continue
            newtokens.extend(sub_tokens)

    dot_tokens = [token for token in newtokens if (token[0] == "." or token[-1] == ".") and (len(token)>1)]
    if len(dot_tokens) > 0 :print(dot_tokens)
    
    return " ".join(newtokens).replace(' . . ',' . ')



'''
def calc_review_len(x):
#     tokens = [token for token in nlp(x)]
#     print(tokens)
#     print([len(t) for t in tokens])
#     return len(tokens)
    return len(x.split(" "))
key_df = deepcopy(orign_key_df)
key_df['lemm_review'] = key_df['lemm_review'].apply(compose_review)
key_df['lemm_review_len'] = key_df['lemm_review'].apply(calc_review_len)
key_df.head()

'''
def bert_compose_review(x):
    x = eval(x)
    review_sents = deepcopy(x)
    total_tokens = []
    for sent in review_sents:
        sent = sent.replace('\n','[SEP]')
        tokens = [str(token) for token in sent.split(" ") if (" " not in str(token)) ]
        newtokens = []
        for token in tokens:
            if (isnumber(token) or len(token) == 1 or token == ".")and (token not in alphbet_stopword):
                newtokens.append(token)
            else:
                token = token.replace("."," . ")
                sub_tokens = token.split(" ")
                sub_tokens = [t for t in sub_tokens if t != "" and t not in alphbet_stopword]
                newtokens.extend(sub_tokens)
        newtokens = newtokens + ['[SEP]']        
        total_tokens.extend(newtokens)
    total_tokens = ['[CLS]'] + tokenizer.tokenize(" ".join(total_tokens))    
    return " ".join(total_tokens)

'''
# def compose_review(bert_review):
#     return bert_review.replace('[CLS] ','').replace('[SEP] ','')
'''
''''''

''

In [11]:
orign_key_df.head()

,Unnamed: 0,review_ID,review,summary,big_categories,small_categories,lemm_review,lemm_summary,lemm_review_len,lemm_summary_len,overall,vote,total_keyword,FOP_sents,total_mention_features,bert_review,bert_summary,bert_review_len,bert_summary_len
0,0,1356566400,this book was almost as small a pamphlet . for...,too small .,Electronics,Camera Cases,this book was almost as small a pamphlet . \nf...,<s> too small </s>,45,4,1,2,"pamphlet as small,pamphlet small",this book was almost as small a pamphlet .,small pamphlet book gift expect intend emphasi...,,,0,0
1,1,1325721600,this book is a collectors item and i love they...,wonderful book a keepsake,Electronics,Camera Cases,this book is a collector item and i love they ...,<s> wonderful book keepsake </s>,54,5,5,9,"item great,item wonderful,quality great,nostal...",\ngreat quality though and a wonderful nostalg...,collector item book high copy price price buy ...,,,0,0
2,2,951782400,i would like to say the the cannon elph37oz is...,elph 37oz is the best .,Electronics,APS Cameras,i would like to say the the cannon elph37 oz i...,<s> elph 37 oz is the best </s>,58,8,5,51,"money extra,camera cheap,camera inferior",\npay the extra money and get this one rather ...,money cannon camera decide buy picture camera ...,,,0,0
3,3,985910400,i have had and used q 370z for over a year and...,a good all around aps camera,Electronics,APS Cameras,i have have and use q 370z for over a year and...,<s> good all around aps camera </s>,137,7,5,13,"light low,eye red room increase slrs lense,len...",\nyes i sometimes get red eye in low light but...,year photographer snapshot lens eye close flas...,,,0,0
4,4,967075200,i used this camera once and it took relatively...,terrible camera worse customer service,Electronics,APS Cameras,i use this camera once and it take relatively ...,<s> terrible camera worse customer service </s>,94,7,1,48,memory internal camera service camera send mon...,\nhowever on my next trip the camera internal ...,picture camera memory camera trip malfunction ...,,,0,0


# Embedding/word2Vec

In [12]:
sentences = [] # total sentence
for idx in tqdm(range(len(orign_key_df))):
    series = orign_key_df.iloc[idx]
    data_dict = series.to_dict()
    lemm_review_sents , lemm_summary  = data_dict['lemm_review'],data_dict['lemm_summary'] 
    try:
        lemm_review_sents = eval(lemm_review_sents)
        for sent in lemm_review_sents:
            sent_tokens = sent.split(" ")
            tokens = [str(token) for token in sent.split() if (" " not in str(token))and (str(token) == '.' or str(token).isalpha())]
    #         tokens = [str(token) for token in sent.split() if (" " not in str(token))]
            sentences.append(tokens)   
        
            dot_tokens = [token for token in tokens if (token[0] == "." or token[-1] == ".") and (len(token)>1)]
            if len(dot_tokens) > 0 :print(dot_tokens)
            
        sentences.append([t for t in lemm_summary.split(" ") if t not in ["<s>" , "</s>"]])
        
        dot_tokens = [token for token in lemm_summary.split(" ") if (token[0] == "." or token[-1] == ".") and (len(token)>1)]
        if len(dot_tokens) > 0 :print(dot_tokens)
    except Exception as e:
        continue
        
    
print('word2Vec training sentence finished...')

100%|██████████| 165310/165310 [00:38<00:00, 4320.89it/s]

word2Vec training sentence finished...


In [13]:
len(sentences)

0